In [4]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer

In [2]:
text = '해보지 않으면 해낼 수 없다'

result = text_to_word_sequence(text)
print(f'Original: {text}')
print(f'Tokenization: {result}')

Original: 해보지 않으면 해낼 수 없다
Tokenization: ['해보지', '않으면', '해낼', '수', '없다']


In [5]:
# Tokenizer
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화합니다.',
        '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
        '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.']
token = Tokenizer()
token.fit_on_texts(docs)

print(f"Word Count: {token.word_counts}")

Word Count: OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])


In [6]:
print(f"Sentence Count: {token.document_count}")

Sentence Count: 3


In [7]:
print(f"How many sentences each words are included in: {token.word_docs}")

How many sentences each words are included in: defaultdict(<class 'int'>, {'각': 1, '텍스트의': 2, '먼저': 1, '단어를': 1, '토큰화합니다': 1, '나누어': 1, '단어로': 1, '딥러닝에서': 2, '토큰화해야': 1, '인식됩니다': 1, '토큰화한': 1, '수': 1, '결과는': 1, '있습니다': 1, '사용할': 1})


In [8]:
print(f"index for each word: {token.word_index}")

index for each word: {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화합니다': 7, '단어로': 8, '토큰화해야': 9, '인식됩니다': 10, '토큰화한': 11, '결과는': 12, '사용할': 13, '수': 14, '있습니다': 15}


# One Hot Encoding

In [9]:
text = "오랫동안 꿈꾸는 이는 그 꿈을 닮아간다"
token = Tokenizer()
token.fit_on_texts([text])
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [10]:
x = token.texts_to_sequences([text])
print(x)

[[1, 2, 3, 4, 5, 6]]


In [11]:
from tensorflow.keras.utils import to_categorical

word_size = len(token.word_index)+1
x = to_categorical(x, num_classes=word_size)

print(x)

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


# Embedding

In [24]:
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.models import Sequential
from numpy import array

docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요",
        "추천하고 싶은 영화입니다","한번 더 보고싶네요",
        "글쎄요","별로예요","생각보다 지루하네요",
        "연기가 어색해요","재미없어요"]
classes = array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)



{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [19]:
x = token.texts_to_sequences(docs)
x

[[1, 2],
 [3],
 [4, 5, 6, 7],
 [8, 9, 10],
 [11, 12, 13],
 [14],
 [15],
 [16, 17],
 [18, 19],
 [20]]

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_x = pad_sequences(x, 4)
padded_x

array([[ 0,  0,  1,  2],
       [ 0,  0,  0,  3],
       [ 4,  5,  6,  7],
       [ 0,  8,  9, 10],
       [ 0, 11, 12, 13],
       [ 0,  0,  0, 14],
       [ 0,  0,  0, 15],
       [ 0,  0, 16, 17],
       [ 0,  0, 18, 19],
       [ 0,  0,  0, 20]], dtype=int32)

In [25]:
from pickletools import optimize


word_size = len(token.word_index)+1

model = Sequential()
model.add(Embedding(word_size, 8, input_length = 4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)
print(f'Accuracy: {model.evaluate(padded_x, classes)[1]}')

Epoch 1/20


2022-08-20 00:26:27.892321: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-20 00:26:28.226505: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 0.6984 - accuracy: 0.4000
Epoch 2/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6966 - accuracy: 0.4000
Epoch 3/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6947 - accuracy: 0.4000
Epoch 4/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6929 - accuracy: 0.4000
Epoch 5/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6910 - accuracy: 0.4000
Epoch 6/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6892 - accuracy: 0.4000
Epoch 7/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6873 - accuracy: 0.5000
Epoch 8/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6855 - accuracy: 0.5000
Epoch 9/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6836 - accuracy: 0.6000
Epoch 10/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6818 - accuracy: 0.6000
Epoch 11/20
1/1 [==============================] 

2022-08-20 00:26:30.337038: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
